# Exploratory Data Analysis for Final Project

In this assignment, your task is to put your new skils to use in the form of an open-ended, exploratory data analysis. In particular, I'm looking for you to demonstrate your ability to use the programmatic tools we've been learning to access data, manipulate it, and visualize some findings while working towards the ultimate goal of developing a final project proposal. Please include the following in your assignment:

  1. A brief summary of the topic (a few sentences)
  2. Access 2 or more datasets, at least one of them using an API (if you are not using any data from an API as part of your project yet, don’t worry about it, and just do this to get some exercise using APIs. You might use it later).
  3. Demonstrate the use of Pandas operations to filter out missing data and/or outliers.
  4. Demonstrate your capacity to use some of the "group-by" operations to produce pivot tables or statistical summaries of your data.
  5. Use Matplotlib or Seaborn to produce 2-3 data visualizations of your data to both explore the data and highlight any notable patterns.
  6. Include a short written analysis of your interpretation of the data.
  7. In a few paragraphs, describe the research question you intend to investigate in your final project, and the plan for the data analysis you intend to perform.

Note that this exercise is intended to help you formulate your project topic. But it is not a binding contract. Your project will most likely evolve over the rest of the semester. So use this as an opportunity to be creative, throw some ideas against the wall and see what sticks. I will release the final project guidelines shortly. In the meantime, dig in!

And as always, please submit this assignment both as a PR on GitHub along with the URL of your PR on bCourses.

1. I want to demonstrate the correlation between different types of power plants in California and demographics (race, ethnicity, income). I will be using California Energy Commission (CEC) power plant data and Census demographic information.

In [32]:
import pandas as pd
import json    
import requests
import pprint
from datetime import datetime
pp = pprint.PrettyPrinter

In [4]:
# API of CEC's power plant location dataset
endpoint_url = "https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Power_Plant/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

In [31]:
# Extract data
response = requests.get(endpoint_url)
data = json.loads(response.text)
data
powerplants = data['features']
powerplants[0]

{'type': 'Feature',
 'id': 1865,
 'geometry': {'type': 'Point',
  'coordinates': [-119.56790556527, 36.1371746042943]},
 'properties': {'OBJECTID': 1865,
  'CECPlantID': 'S0335',
  'PlantName': 'Corcoran 2 Solar LLC CED',
  'Retired_Plant': 0,
  'OperatorCompanyID': 'CED California Holdings LLC',
  'County': 'Kings',
  'Capacity_Latest': 19.8,
  'Units': '1',
  'PriEnergySource': 'SUN',
  'StartDate': 1433894400000}}

In [47]:
# Transform JSON to tabluar DF
d = {
    'CECID': [p['properties']['CECPlantID'] for p in powerplants],
    'PlantName': [p['properties']['PlantName'] for p in powerplants],
    'Retired': [p['properties']['Retired_Plant'] for p in powerplants],
    'Operator': [p['properties']['OperatorCompanyID'] for p in powerplants],
    'County': [p['properties']['County'] for p in powerplants],
    'Capacity': [p['properties']['Capacity_Latest'] for p in powerplants],
    'Units': [p['properties']['Units'] for p in powerplants],
    'Energy Source': [p['properties']['PriEnergySource'] for p in powerplants],
    'Date of Operation': [p['properties']['StartDate'] for p in powerplants],
    'Latitude': [p['geometry']['coordinates'][0] for p in powerplants],
    'Longitude': [p['geometry']['coordinates'][1] for p in powerplants],
}
df = pd.DataFrame.from_dict(d)
# UNIX to YMD
df['Date of Operation'] = df['Date of Operation'].astype("datetime64[ms]")

In [48]:
df

,CECID,PlantName,Retired,Operator,County,Capacity,Units,Energy Source,Date of Operation,Latitude,Longitude
0,S0335,Corcoran 2 Solar LLC CED,0,CED California Holdings LLC,Kings,19.8,1,SUN,2015-06-10,-119.567906,36.137175
1,S0520,Corcoran 3 Solar,0,CED California Holdings LLC,Kings,20.0,Unit 1,SUN,2016-02-11,-119.579725,36.144324
2,C0007,Hanford - Retired October 2011,1,Hanford LP,Kings,24.0,GEN 1,PC,1990-09-01,-119.648404,36.269645
3,G0832,Hanford Energy Park Peaker,0,"MRP San Joaquin Energy, LLC",Kings,92.0,"1, 2",NG,2001-09-01,-119.647450,36.270313
4,S0608,Exeter Solar,0,Tulare PV I LLC,Tulare,3.5,ES,SUN,2014-02-12,-119.128338,36.266303
...,...,...,...,...,...,...,...,...,...,...,...
1797,S0571,Lemoore PV 1 LLC,0,Lemoore PV 1 LLC,Kings,1.5,Unit 1,SUN,2016-01-29,-119.797348,36.257775
1798,S0225,Guernsey Solar Station,0,Pacific Gas & Electric (PG&E),Kings,20.0,1,SUN,2013-09-18,-119.650488,36.164281
1799,S0249,Corcoran LLC CED,0,CED California Holdings LLC,Kings,20.0,1,SUN,2013-08-13,-119.573740,36.142176
1800,S0317,Corcoran Irrigation District Solar LLC,0,Onward Energy,Kings,20.0,1,SUN,2014-12-22,-119.581404,36.141331


In [24]:
lat = 37.761216
lon = -122.396965
url = 'https://geo.fcc.gov/api/census/block/find?latitude={0}&longitude={1}&format=json'.format(lat, lon)
response = requests.get(url)
data = response.json()
data

{'Block': {'FIPS': '060310013002003',
  'bbox': [-119.572017, 36.13663, -119.563119, 36.137939]},
 'County': {'FIPS': '06031', 'name': 'Kings County'},
 'State': {'FIPS': '06', 'code': 'CA', 'name': 'California'},
 'status': 'OK',
 'executionTime': '0'}

In [25]:
# define parameters of our API query
acs_var = 'B01001_001E'  # total pop
state = '06'  # CA
counties = ['001', '075']  # Alameda and SF
year = 2018

# format the URL
counties_str = ','.join(counties)
url = "https://api.census.gov/data/{0}/acs/acs5?get={1}&for=county:{2}&in=state:{3}".format(
    year, acs_var, counties_str, state)

# execute the HTTP request
res = requests.get(url)
pd.DataFrame(res.json())

,0,1,2
0,B01001_001E,state,county
1,870044,06,075
2,1643700,06,001


In [27]:
import os
from census import Census
from us import states

# I store my API key stored as an environment variable rather than a .json file
census_key = os.getenv("CENSUS_API")  

c = Census(census_key, year=year)
res = c.acs5.get((
    'NAME', acs_var), {
    'for': 'county:{}'.format(counties_str),
    'in': 'state:{}'.format(states.CA.fips)
})

pd.DataFrame(res)

,NAME,B01001_001E,state,county
0,"San Francisco County, California",870044.0,06,075
1,"Alameda County, California",1643700.0,06,001


In [28]:
acs_df = pd.DataFrame()

for year in range(2010, 2018):
    c = Census(census_key, year=year)
    res = c.acs1.get((
        'NAME', acs_var), {
        'for': 'county:{}'.format(counties_str),
        'in': 'state:{}'.format(states.CA.fips)
    })

    year_df = pd.DataFrame(res)
    year_df['year'] = year
    acs_df = pd.concat((acs_df, year_df), ignore_index=True)